# Softmax 回归：MNIST 数据集

## MNIST 数据集的下载和导入

[MNIST 数据集](http://yann.lecun.com/exdb/mnist/) 是一个手写数字组成的数据集，现在被当作一个机器学习算法评测的基准数据集。

这是一个下载并解压数据的脚本：

In [1]:
%%file download_mnist.py
import os
import os.path
import urllib
import gzip
import shutil

if not os.path.exists('mnist'):
    os.mkdir('mnist')

def download_and_gzip(name):
    if not os.path.exists(name + '.gz'):
        urllib.urlretrieve('http://yann.lecun.com/exdb/' + name + '.gz', name + '.gz')
    if not os.path.exists(name):
        with gzip.open(name + '.gz', 'rb') as f_in, open(name, 'wb') as f_out:
            shutil.copyfileobj(f_in, f_out)
            
download_and_gzip('mnist/train-images-idx3-ubyte')
download_and_gzip('mnist/train-labels-idx1-ubyte')
download_and_gzip('mnist/t10k-images-idx3-ubyte')
download_and_gzip('mnist/t10k-labels-idx1-ubyte')

Overwriting download_mnist.py


可以运行这个脚本来下载和解压数据：

In [2]:
%run download_mnist.py

使用如下的脚本来导入 MNIST 数据，源码地址：

https://github.com/Newmu/Theano-Tutorials/blob/master/load.py

In [3]:
%%file load.py
import numpy as np
import os

datasets_dir = './'

def one_hot(x,n):
	if type(x) == list:
		x = np.array(x)
	x = x.flatten()
	o_h = np.zeros((len(x),n))
	o_h[np.arange(len(x)),x] = 1
	return o_h

def mnist(ntrain=60000,ntest=10000,onehot=True):
	data_dir = os.path.join(datasets_dir,'mnist/')
	fd = open(os.path.join(data_dir,'train-images-idx3-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	trX = loaded[16:].reshape((60000,28*28)).astype(float)

	fd = open(os.path.join(data_dir,'train-labels-idx1-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	trY = loaded[8:].reshape((60000))

	fd = open(os.path.join(data_dir,'t10k-images-idx3-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	teX = loaded[16:].reshape((10000,28*28)).astype(float)

	fd = open(os.path.join(data_dir,'t10k-labels-idx1-ubyte'))
	loaded = np.fromfile(file=fd,dtype=np.uint8)
	teY = loaded[8:].reshape((10000))

	trX = trX/255.
	teX = teX/255.

	trX = trX[:ntrain]
	trY = trY[:ntrain]

	teX = teX[:ntest]
	teY = teY[:ntest]

	if onehot:
		trY = one_hot(trY, 10)
		teY = one_hot(teY, 10)
	else:
		trY = np.asarray(trY)
		teY = np.asarray(teY)

	return trX,teX,trY,teY

Overwriting load.py


## softmax 回归

`Softmax` 回归相当于 `Logistic` 回归的一个一般化，`Logistic` 回归处理的是两类问题，`Softmax` 回归处理的是 `N` 类问题。

`Logistic` 回归输出的是标签为 1 的概率（标签为 0 的概率也就知道了），对应地，对 N 类问题 `Softmax` 输出的是每个类对应的概率。

具体的内容，可以参考 `UFLDL` 教程：

http://ufldl.stanford.edu/wiki/index.php/Softmax%E5%9B%9E%E5%BD%92

In [4]:
import theano
from theano import tensor as T
import numpy as np
from load import mnist

Using gpu device 0: GeForce GTX 850M


我们来看它具体的实现。

这两个函数一个是将数据转化为 `GPU` 计算的类型，另一个是初始化权重：

In [5]:
def floatX(X):
    return np.asarray(X, dtype=theano.config.floatX)

def init_weights(shape):
    return theano.shared(floatX(np.random.randn(*shape) * 0.01))

`Softmax` 的模型在 `theano` 中已经实现好了：

In [6]:
def model(X, w):
    return T.nnet.softmax(T.dot(X, w))

导入数据：

In [7]:
trX, teX, trY, teY = mnist(onehot=True)

定义变量，并初始化权重：

In [8]:
X = T.fmatrix()
Y = T.fmatrix()

w = init_weights((784, 10))

定义模型输出和预测：

In [9]:
py_x = model(X, w)
y_pred = T.argmax(py_x, axis=1)

损失函数为多类的交叉熵，这个在 `theano` 中也被定义好了：

In [10]:
cost = T.mean(T.nnet.categorical_crossentropy(py_x, Y))
gradient = T.grad(cost=cost, wrt=w)
update = [[w, w - gradient * 0.05]]

编译 `train` 和 `predict` 函数：

In [11]:
train = theano.function(inputs=[X, Y], outputs=cost, updates=update, allow_input_downcast=True)
predict = theano.function(inputs=[X], outputs=y_pred, allow_input_downcast=True)

迭代 100 次，看测试集的正确率：

In [12]:
for i in range(100):
    for start, end in zip(range(0, len(trX), 128), range(128, len(trX), 128)):
        cost = train(trX[start:end], trY[start:end])
    print i, np.mean(np.argmax(teY, axis=1) == predict(teX))

0 0.8844
1 0.8984
2 0.9053
3 0.9076
4 0.9094
5 0.9104
6 0.9121
7 0.9135
8 0.9151
9 0.9156
10 0.9163
11 0.9166
12 0.9167
13 0.9174
14 0.9179
15 0.9182
16 0.9185
17 0.9188
18 0.9185
19 0.9187
20 0.9196
21 0.92
22 0.9205
23 0.9201
24 0.9203
25 0.9202
26 0.9205
27 0.9207
28 0.921
29 0.9211
30 0.9212
31 0.9217
32 0.9218
33 0.9217
34 0.922
35 0.9219
36 0.9218
37 0.9217
38 0.9219
39 0.9217
40 0.9219
41 0.9222
42 0.9223
43 0.9224
44 0.9224
45 0.9223
46 0.9225
47 0.9225
48 0.9225
49 0.9225
50 0.9226
51 0.9227
52 0.9228
53 0.923
54 0.923
55 0.923
56 0.9231
57 0.9233
58 0.9235
59 0.9236
60 0.9237
61 0.9239
62 0.924
63 0.9242
64 0.9242
65 0.924
66 0.9241
67 0.9241
68 0.9241
69 0.924
70 0.9243
71 0.9243
72 0.9244
73 0.9243
74 0.9242
75 0.924
76 0.9243
77 0.9245
78 0.9246
79 0.9245
80 0.9245
81 0.9246
82 0.9245
83 0.9244
84 0.9243
85 0.9245
86 0.9246
87 0.9246
88 0.9247
89 0.9248
90 0.9249
91 0.9249
92 0.9247
93 0.9248
94 0.9248
95 0.9248
96 0.9248
97 0.9248
98 0.9248
99 0.9246
